# Segmenter (Transformer for Semantic Segmentation)  
- 논문 리뷰

## Abstract  
- Segmentation task는 이미지 패치를 다루기에는 애매모호하며 context한 정보를 필요로 한다  
- 이 논문에서 transformer를 사용한 semantic segmentation model인 Segmenter를 소개한다  
- 이를 위해, 모델은 이미지 패치에 해당하는 output embedding에 의존하며 decoder를 통해 class label을 만든다  
- linear decoder도 충분히 좋은 성능을 얻을 수 있지만 더 좋은 성능을 가지는 mask transformer decoder를 사용한다  
- 논문에서 다양한 파라미터의 영향을 보여주기 위해 대규모의 연구를 진행했으며 그 결과로 특히 큰 모델과 작은 패치 사이즈를 사용한 것이 더 성능이 좋다고 소개했다  
- Segmenter는 ADE20K와 Pascal Context dataset에서 SOTA 모델의 성능을 능가하며 cityscapes dataset에 대해서는 충분히 경쟁력 있는 성능을 보여주고 있다  


## Introduction  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/figure1.JPG?raw=true" width=600>

- semantic segmentation은 일반적으로 convolution의 encoder-decoder의 architecture를 따른다  
- 그러나 convolution filter의 local 특성은 이미지의 global한 정보의 access를 제한한다  
- global한 정보는 segmentation에서 굉장히 중요하며 segmentation에서 local patch의 labeling은 global image context에 의존한다  
- 그러나 ViT를 활용한 더 hard한 vision task에서는 여전히 convolution에 의존하는 경향이 있으며 local interaction에 편향된다  
- 이를 해결하기 위해 특수 레이어를 광범위하게 사용하면 convolution architecture의 한계를 알 수 있다  
- 이 한계를 극복하기 위해 우리는 semantic segmentation problem을 sequence-to-sequence 문제로 공식화하며 모델의 모든 stage에서 contexual 정보를 조절하기 위해 transformer architecture를 사용한다  
- 이러한 디자인 덕분에 transformer는 scene의 element 사이의 global interaction을 캡처할 수 있으며 inductive prior가 내장되어 있지 않다  
- 우리는 이미지를 패치로 자른 후 linear patch embedding을 transformer encoder의 input으로 처리한다  
- encoder에서 생성된 token의 context한 sequence는 transformer decoder에서 픽셀마다 클래스 score로 upsampling된다  
- decoding을 위해 우리는 class score에 대해 patch embedding의 point-wise linear mapping 또는 transformer 기반 decoding 체계를 고려한다  
- 이때 decoding은 학습 가능한 클래스 embedding은 클래스 mask를 생성하는 patch token과 함께 처리된다  
- Segmenter는 단순하고 유연하며 빠르게 유지되면서 우수한 결과를 달성한다  
- 특히 작은 patch 사이즈의 large model을 사용할 때 가장 좋은 모델은 ADE20K dataset에서 53.63%의 mean IoU를 달성하며 기존 conv 기반 SOTA 모델보다 5.3% 이상의 좋은 성능을 낸다  
- 본 논문의 연구를 4가지로 요약하면 다음과 같다  
    - 1. convolution 없이 ViT 기반 semantic segmentation model을 제안한다  
        - 이는 FCN 기반 모델들의 성능을 능가하며 context한 정보들을 캡처한다  
    - 2. precision과 run-time 사이의 trade-off를 극복하는 다양한 level의 해상도를 가진 모델을 제시한다  
    - 3. linear baseline을 능가하고 일반적인 image segmentation task를 수행하기 위해 확장할 수 있는 class mask를 생성하는 transformer 기반 decoder를 제안한다  
    - 4. 우리의 접근법이 ADE20K와 Pascal Context dataset에서 SOTA를 달성하고 cityscapes dataset에서 충분히 경쟁력 있는 성능을 낸다는 것을 보여준다

## Related Work  
- 평소에 논문을 읽을 때 related work 파트는 잘 읽지 않지만 ViT는 vision task에서 최신 기법인만큼 최신 연구 동향은 어떻게 되는지 파악하기 위해 읽었다  
- 다음 모델들이 정확히 어떤 architecture로 이루어졌는지는 잘 모르지만 vision transformer에서 꽤나 중요한 모델들이라고 생각하며 추후에 관련 논문도 읽어볼 예정이다  

#### Transformers for vision  
**DeiT**  
- ViT는 large scale dataset에서의 사전 훈련이 필요하지만 DeiT는 token 기반 distillation strategy를 제안하고 teacher로 CNN을 사용해 ImageNet-1K dataset에 대해 훈련된 경쟁력 있는 vision transformer를 얻는다  
- DeiT의 경우, ViT는 사전 훈련 비용이 많이 든다는 것에 문제를 제기하며 효율적으로 학습을 할 수 있도록 제안했다  

**SETR**  
- SETR의 경우 ViT backbone과 standard CNN decoder를 사용한다  

**Swin Transformer**  
- layer와 UpperNet 사이에서 shift되는 local window로 구성된 ViT를 변형해 피라미드 FCN decoder처럼 사용한다  
- Swin Transformer의 경우, 이를 기반으로 한 Semantic Segmentation 논문이 나온 걸로 알고 있다  
- 그래서 Swin Transformer는 나중에 꼭 읽어봐야 할 것 같다  
  
**Our**  
- Segmenter는 ViT backbone를 베이스로 하며 DETR에서 영감을 받아 mask decoder를 설계했다  

## Our approach: Segmenter  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/figure2.JPG?raw=true" width=800>

- Segmenter는 fully transformer 기반 encoder-decoder architecture로 이루어져 있으며 patch embedding의 sequence가 pixel level class로 매핑된다  
- Segmenter는 end-to-end로 훈련되며 픽셀별로 cross entropy loss를 사용한다  
- inference time에는 픽셀마다 1개의 클래스를 얻기 위해 upsampling한 후 argmax를 적용한다  

### Encoder  
- 이미지 $\textbf{x}\in \mathcal{R}^{H\times W\times C}$는 patch sequence로 split되며 $\textbf{x}\in\mathcal{R}^{N\times P^2\times C}$로 dimension이 변한다  
- 이때 $P\times P$는 patch size, $N=HW/P^2$은 patch 갯수, $C$는 채널 수다  
- 각 patch는 $1D$ vector로 flatten되고 patch embedding의 sequence를 생성하기 위해 linear projection을 통과한다  
- 위치 정보를 캡처하기 위해 학습 가능한 position embedding을 sequence에 추가한다  
- 그러면 최종 sequence는 $\textbf{z}_0=\textbf{x}_9+pos$가 된다  
- transformer encoder는 token $\textbf{z}_0$의 sequence에 적용되어 $\textbf{z}_L\in\mathcal{R}^{N\times
D}$의 sequence를 생성한다  
- transformer encoder는 MSA block과 MLP block으로 이루어져있으며 LN은 block 이전에, residual connection은 block 이후에 적용된다  
$$\textbf{a}_{i-1}=MSA\left(LN\left(\textbf{z}_{i-1}\right)\right)+\textbf{z}_{i-1}$$
$$\textbf{z}_i=MLP\left(LN\left(\textbf{a}_{i-1}\right)\right)+\textbf{a}_{i-1}$$  
- self-attention mechanism은 token을 중간 표현, query, key, value에 매핑하는 3개의 point-wise linear layer로 구성된다  
- self-attention은 다음과 같이 계산된다  
$$MSA\left(Q, K, V\right)=softmax\left(\frac{QK^T}{\sqrt{d}}\right)V$$  
- transformer encoder는 embedded patch의 input sequence $\textbf{z}_0=\left[z_{0,1},\cdots,\textbf{z}_{0,N}\right]$를 $\textbf{z}_L=\left[z_{L,1},\cdots,\textbf{z}_{L,N}\right]$으로 매핑하며 이때 $\textbf{z}_L$은 decoder가 사용하는 풍부한 semantic 정보를 포함하는 contexual된 encoding sequence다  

### Decoder  
- encoding된 sequence $\textbf{z}_L\in\mathcal{R}^{N\times D}$는 segmentation map $\textbf{s}\in\mathcal{R}^{H\times W\times K}$로 decoding되며 이때 $K$는 클래스 갯수다  
- decoder는 encoder에서 생긴 patch-level encoding을 patch-level class score로 매핑하기 위해 학습한다  
- 그리고 이 patch-level class score는 bilinear interpolation으로 upsampling된다  

#### linear decoder와 mask transformer를 비교한 단락  
**Linear Decoder**  
- point-wise linear decoder는 patch-level class logit $\textbf{z}_{lin}\in\mathcal{R}^{N\times K}$를 생성하기 위해 patch encoding $\textbf{z}_L\in\mathcal{R}^{N\times D}$에 적용된다  
- 그러면 이 sequence는 $2D$ feature map으로 reshape되며 original size에 맞춰 bilinear interpolation으로 upsampling된다  
- 그리고 softmax를 적용한 후 최종 segmentation map을 얻는다  

**Mask Transformer**  
- mask transformer는 transformer 기반 decoder이며 $K$개의 학습 가능한 class embedding $\left[cls_1,\cdots,cls_K\right]\in\mathcal{R}^{K\times D}$을 소개한다  
- 각 class embedding은 랜덤으로 초기화되며 하나의 semantic class로 할당된다  
- class embedding cls는 decoder에 의해 patch embedding과 함께 처리된다  
- mask transformer는 L2-normalized patch embedding $\textbf{z}_M'\in\mathcal{R}^{N\times D}$와 class embedding $\textbf{c}\in\mathcal{R}^{K\times D}$ 사이의 내적을 계산해 K mask를 계산한다  
- 그러면 각 mask sequence는 $\textbf{s}_{mask}\in\mathcal{R}^{H/P\times W/P\times K}$의 2D로 reshape되며 original input size에 맞춰 bilinear로 upsampling된다  
- 그리고 softmax를 적용한 후 pixel-wise class score를 얻는다  

- 순수한 transformer architecture를 사용하고 patch-level encoding을 조절하면서 우리는 decoding phase 동안 patch와 class embedding을 공동으로 처리하는 간단한 접근 방식을 제안했다  
- 이 논문에서는 semantic segmentation을 다루지만 mask transformer는 class embedding을 object embedding으로 대체해 panoptic segmentation을 수행하도록 조정할 수 있다  

## Experimental results  
### Datasets and metrics  

**ADE20K**  
- class: 150  
- train set: 20210  
- valid set: 2000  
- test set: 3352  

**Pascal Context**  
- class: 60  
- train set: 4996  
- valid and test set: 5104  

**Cityscapes**  
- class: 19  
- train set: 2975  
- valid set: 500  
- test set: 1525  

**Metrics**  
- mean IoU  

### Implementation details  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table1.JPG?raw=true">

**Transformer models**  
- encoder에 여러 변형을 적용해 Tiny, Small, Base, Large 모델과 DeiT 모델을 사용했다  
- MSA block의 head size는 64로 고정시키고 head number는 token size를 head size로 나눈 값이며 MLP의 hidden size는 token size의 4배이다  
- 다른 해상도의 이미지와 $8\times8, 16\times16, 32\times32$의 다른 patch size로 실험했다  
- Segmenter model은 ImageNet에서 사전 훈련되었으며 ViT는 augmentation과 함께 ImageNet-21K에서 사전훈련했다  
- 그리고 DeiT는 ImageNet-1K에서 사전훈련했다  
- 이미지 분류 라이브러리 timm과 google에서 공개한 모델을 사용했다  
- 모델은 해상도 224에서 사전 훈련되고 ViT-B/8을 제외하고 384의 해상도의 ImageNet-1K에서 fine-tuning을 진행했다  
**Data augmentation**  
- (0.5, 2.0)의 scale로 random resize, random left-right flipping을 진행한다  
- random cropping의 경우, ADE20K에서 $512\times512$, Pascal Context에서 $480\times480$, Cityscapes에서 $768\times768$의 size로 crop을 진행한다  

**Optimization**  
- loss function은 cross entropy loss를 사용한다  
- optimizer로는 SGD를 사용하며 weight decay는 0으로 설정한다  
- learning rate는 poly learning rate decay $\gamma=\gamma_0\left(1-\frac{N_{iter}}{N_{total}}\right)^{0.9}$를 사용한다  
- ADE20K에서 초기 learning rate는 $10^{-3}$, 160K iterations, batch size는 8을 사용한다  
- Pascal Context에서 초기 learning rate는 $10^{-3}$, 80K iterations, batch size는 16을 사용한다  
- Cityscapes에서 초기 learning rate는 $10^{-2}$, 80K iterations, batch size는 8을 사용한다  

### Ablation study  
- 이 section에서는 ADE20K validation set을 가지고 다양한 접근 방식으로 각각 어떤 성능을 내는지 실험을 한 결과를 보여준다  
- model regularization, model size, patch size를 다양하게 적용해 모델의 성능을 비교한다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table2.JPG?raw=true" width=500>

**Regularization**  
- 2가지 regularization과 dropout, stochastic depth를 비교하고 stochastic depth가 segmentation에 대해 어떻게 학습하는지 보여준다  
- CNN의 경우, regularizer의 역할도 수행하는 batch norm에 의존한다  
- 반면, transformer는 일반적으로 dropout과 layer norm으로 구성된다  
- dropout은 input node를 랜덤으로 무시하고 stochastic depth는 forward pass 동안 block을 랜덤으로 무시한다  
- table2에서 dropout rate는 0, stochastic depth rate는 0.1인 경우가 가장 성능이 좋다  
- dropout은 단독으로 또는 stochastic depth와 같이 사용할 경우 모델의 성능을 저하시킨다  
- 그래서 모델은 dropout 없이 stochastic depth로만 학습을 시킨다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table3.JPG?raw=true" width=500>

**Transformer size**  
- patch size를 16으로 고정시키고 다양한 레이어 수와 token size에 대해 transformer의 성능을 연구한다  
- token dimension을 두 배로 늘리면 (Seg-B/16 $\rightarrow$ Seg-L/16) 성능이 2.69% 향상된다  
- 레이어 수를 두 배로 늘리면 (Seg-B/16 $\rightarrow$ Seg-L/16) 성능이 2.65% 향상된다  
- Seg-L/16 모델의 경우, ADE20K validation set에서 mean IoU 50.71%의 좋은 성능을 낸다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/figure3.JPG?raw=true">

**Patch size**  
- 이미지를 patch sequence로 표현하는 것은 다양한 patch size에 따른 speed와 accuracy 사이의 trade-off가 존재한다  
- patch size를 늘리면 이미지의 representation이 더 coarse하지만 sequence의 길이가 짧아지며 더 빨라진다  
- 우리는 patch size가 semantic segmentation의 성능에 중요한 요소라는 것을 관찰했다  
- table3에서 Seg-B 모델에 대해 patch size를 32에서 16으로 줄일 때 성능이 5% 향상된 것을 볼 수 있다  
- 또한 patch size를 8까지 줄일 때 가장 성능이 좋은 것을 볼 수 있다  
- 이를 통해 알 수 있는 것은 patch size를 줄이면 모델의 성능이 더 좋아지지만 sequence의 길이가 길어지면서 계산량이 늘어나는 문제가 있다는 것이다  
- 만약 계산적인 측면을 고려하지 않는다면, ViT-L/8 모델이 가장 좋은 성능을 낼 것이라고 예측할 수 있다  
- figure3에서 patch size에 따른 segmentation map을 보여주고 있다  
- patch size가 32인 경우, segmentation이 global하게 수행되는 경향이 있다  
- patch size를 줄일수록 점차 detail한 segmentation map이 생성된다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table4.JPG?raw=true" width = 500>

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/figure8.JPG?raw=true" width = 700>

**Decoder variants**  
- mask transformer를 평가하고 linear decoder와 비교를 할 것이다  
- mask transformer에는 encoder와 동일한 token과 hidden size를 가지고 있다  
- table 4에서 mask transformer는 일관적으로 linear decoder보다 좋은 성능을 보여주고 있다  
- 또한 object의 크기에 따라 다양한 모델의 성능을 살펴봤다  
- 작고 큰 object에 대해 dynamical filter를 사용하는 것이 더 큰 benefit을 얻는다는 것을 살펴봤다  
- mask transformer에 의해 학습된 class embedding은 의미가 있다  
- 즉, figure 8을 보면 유사한 클래스끼리 가까이 있는 것을 알 수 있다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table6.JPG?raw=true" width = 700>

**Transformer versus FCN**  
- table 6에서 FCN 기반 모델과 transformer 모델을 비교했다  
- 우리의 transformer 접근 방식은 CNN 기반 SOTA 모델에 비해 크게 개선되어 global scene understanding을 캡처할 수 있는 transformer의 능력을 강조한다  
- Segmenter는 large instance에 대해 DeepLab보다 좋은 성능을 보여주고 있으며 patch size가 16보다 작거나 같을 경우 전체적으로 더 좋은 성능을 보여주고 있다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/figure4.JPG?raw=true" width = 700>

**Performance**  
- 우리의 모델과 SOTA 모델을 초당 이미지, mIoU 측면에서 비교해보았다  
- FCN 기반 모델보다 transformer가 speed나 mean IoU 측면에서 더 좋은 성능을 보여주고 있다  
- transformer의 경우, 정확도 측면에서 32의 patch size보다 16의 patch size가 더 성능이 좋은 것을 관찰했다  
- Seg/32 모델은 coarse한 segmentation map을 생성하지만 초당 400장으로 ResNet 모델보다 4배 이상 빠르면서 비슷한 성능을 내고 있다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table5.JPG?raw=true" width = 700>

**Dataset size**  
- ViT는 classification task에서 좋은 성능을 얻기 위해 large dataset에서 사전훈련을 했다  
- table 5에서 dataset의 scale에 따라 Segmenter의 성능을 살펴봤다  
- 이는 너무 크지 않은 즉, 충분히 큰 dataset으로도 충분히 좋은 성능을 낸다는 것을 보여준다  

### Comparison with state of the art  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table6.JPG?raw=true" width=500>

**ADE20K**  
- Seg-B+/16 모델은 ImageNet-1K에서 사전 훈련을 했다  
- mask transformer를 추가하면 성능이 약 2% 향상되며 FCN 기반 모델들을 능가한다  
- Seg-L-Mask/16 모델의 경우, 53.63%의 우수한 성능으로 SOTA를 달성했다    

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table7.JPG?raw=true" width=500>

**Pascal Context**
- Seg-B+ 모델은 FCN에 비해 충분히 경쟁력 있는 모델이며 Seg-L/16 모델은 SOTA를 달성했다  
- 특히, mask transformer를 사용하면 linear decoder보다 2.5% 더 좋은 성능을 내며 59.04%의 mIoU를 달성한다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/table8.JPG?raw=true" width=500>
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/Segmenter/figure/figure5.JPG?raw=true" width=800>

**Cityscapes**  
- $768\times 768$의 큰 해상도로 인해 2개의 layer로는 메모리가 부족해 1개의 mask transformer layer만 사용했다  
- Seg-B와 Seg-L 모델 모두 SOTA 모델들과 비교해 경쟁력 있는 성능을 갖췄으며 Seg-L-Mask/16의 경우 81.3%의 mIoU를 달성했다  
- figure 5에서 Segmenter와 DeepLabv3+의 성능을 비교했다  
- DeepLabv3+가 더 detail하게 boundary를 생성하는 반면 Segmenter는 더 큰 instance에서 일관된 label을 제공한다 

## Conclusion  
- 이 논문에서는 순수한 transformer만을 사용해 semantic segmentation model을 제안했다  
- encoder 부분은 ViT로 decoder는 mask transformer로 구축했다  
- 우리는 transformer가 global context를 잘 캡처한다는 것을 관찰했다  
- Decoder의 경우 linear decoder보다 mask transformer가 더 성능이 좋다는 것도 관찰했다  
- 우리가 제안한 Segmenter를 변형해 semantic segmentation 뿐만 아니라 instance, panoptic segmentation task까지 수행할 수 있다